# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "/users/menard/desktop/pandas-challenge/starter_code/pycityschools/resources/schools_complete.csv"
student_data_to_load = "/users/menard/desktop/pandas-challenge/starter_code/pycityschools/resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Calculate the total number of unique schools
school_count = school_data_complete["school_name"].nunique()
school_count

15

In [3]:
# Calculate the total number of students
student_count = school_data_complete["Student ID"].nunique()
student_count

39170

In [4]:
# Calculate the total budget
# Resorting back to school data because each school has their unique budget. When summing from the
# merged data, it will include all of the school budget vs total unique school budget.
total_budget = school_data["budget"].sum()
total_budget

24649428

In [5]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score

78.98537145774827

In [6]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [7]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [8]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
# Testing to see if student ID will give out the same percetage as using student name. Yes, same results
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["Student ID"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [9]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schools": [school_count],
                                "Total Students": [student_count],
                                "Total Budget": [total_budget],
                                "Total Average Match Score": [average_math_score],
                                "Total % of Students Who Passed Math": [passing_math_percentage],
                                "Total Average Reading Score": [average_reading_score],
                                "Total % of Students Who Passed Reading": [passing_reading_percentage],
                                "Total Overall Passing Rate": [overall_passing_rate]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Total Average Match Score,Total % of Students Who Passed Math,Total Average Reading Score,Total % of Students Who Passed Reading,Total Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,74.980853,81.87784,85.805463,65.172326


## School Summary

In [11]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [12]:
# Calculate the total student count
# Since the question is asking use what the total number of students per school is. Think pivot
# table in excel. Everything is sorted per unique value. In order to create that we will need to use
# the groupby syntax in combination with the count to get a count of student names per unique school.
per_school_counts = school_data_complete.groupby(["school_name"]).count()["student_name"]
per_school_counts

school_name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: student_name, dtype: int64

In [13]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

In [14]:
# Calculate the average test scores
# Same concept as above, using school name since we want it per individual school and subject
# scores to calculate the average per student in each unique school. 
per_school_math = school_data_complete.groupby(["school_name"]).mean()["math_score"]
per_school_math
per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]
per_school_reading

school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
Name: reading_score, dtype: float64

In [15]:
# Calculate the number of schools with math scores of 70 or higher
# The code below is a pretty dense code, in translation we are wanting to look at schools with math
# scores woth 70 or higher. In other words, how many students per school have 70 or higher in math.
# We will first be referencing the dataframe, complete school data as it has all the information in
# it. Within that dataframe, we want to look at the math score column to make an arguement (sort of like
# an if statement) if match scores are above 70 then group the schools by their name and then
# count the number of students that have them. (Feels more like a nested loop from VBA)
school_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name").count()["student_name"]
school_passing_math

school_name
Bailey High School       3318
Cabrera High School      1749
Figueroa High School     1946
Ford High School         1871
Griffin High School      1371
Hernandez High School    3094
Holden High School        395
Huang High School        1916
Johnson High School      3145
Pena High School          910
Rodriguez High School    2654
Shelton High School      1653
Thomas High School       1525
Wilson High School       2143
Wright High School       1680
Name: student_name, dtype: int64

In [16]:
# Calculate the number of schools with reading scores of 70 or higher
# NOTE: pay attention to the brackets and parenthesis. They mean two different things. One is for lists, and one is for dataframes
# in this case is indexed.
school_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name").count()["student_name"]
school_passing_reading

school_name
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
Name: student_name, dtype: int64

In [17]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
passing_math_and_reading

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [18]:
# Use the provided code to calculate the passing rates
# The code provided was making a group by calculating when the calculation(s) above had already been done
# the results of math and reading per school have already been defined and therefore why I used them
# in this block of code instead.
# Dividing by two because the math and reading passing rates give us the total overall in respective subject.
# thefore we can get the average when diding by 2
per_school_passing_math = school_passing_math / per_school_counts * 100
per_school_passing_reading = school_passing_reading / per_school_counts * 100
overall_passing_rate = (per_school_passing_math + per_school_passing_reading) / 2

In [19]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({"School Type": school_types,
                                 "Total Students Per School": per_school_counts,
                                 "Total School Budget": per_school_budget,
                                 "Per Student Budget": per_school_capita,
                                 "Average Math Score Per School": per_school_math,
                                 "Average Reading Score Per School": per_school_reading,
                                 "Percent in Passing Math": per_school_passing_math,
                                 "Percent in Passing Reading": per_school_passing_reading,
                                 "Percent Of Passing Rate": overall_passing_rate})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students Per School,Total School Budget,Per Student Budget,Average Math Score Per School,Average Reading Score Per School,Percent in Passing Math,Percent in Passing Reading,Percent Of Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270


## Highest-Performing Schools (by % Overall Passing)

In [21]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
# Pretty straight forward code, having to sort by values within the "Overall % of Passing Rate" 
# calculated above. The ascending is false because if the ascending were positive it would mean that
# it will go from lowest to highest. What they are asking for is highest therefore we want to descend
per_school_summary.sort_values(by='Percent Of Passing Rate',ascending=False).head(5)

,School Type,Total Students Per School,Total School Budget,Per Student Budget,Average Math Score Per School,Average Reading Score Per School,Percent in Passing Math,Percent in Passing Reading,Percent Of Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By % Overall Passing)

In [22]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
# Same concept as the code above.

per_school_summary.sort_values(by="Percent Of Passing Rate", ascending=True).head(5)

,School Type,Total Students Per School,Total School Budget,Per Student Budget,Average Math Score Per School,Average Reading Score Per School,Percent in Passing Math,Percent in Passing Reading,Percent Of Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

In [23]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = school_data_complete.groupby(["school_name"]).mean()
tenth_graders_scores = school_data_complete.groupby(["school_name"]).mean()
eleventh_graders_scores = school_data_complete.groupby(["school_name"]).mean()
twelfth_graders_scores = school_data_complete.groupby(["school_name"]).mean()

# Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores.mean()["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({"9th": ninth_grade_math_scores,
                                     "10th": tenth_grader_math_scores,
                                     "11th": eleventh_grader_math_scores,
                                     "12th": twelfth_grader_math_scores})

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.048432,77.048432,80.43253,77.048432
Cabrera High School,83.061895,83.061895,80.43253,83.061895
Figueroa High School,76.711767,76.711767,80.43253,76.711767
Ford High School,77.102592,77.102592,80.43253,77.102592
Griffin High School,83.351499,83.351499,80.43253,83.351499
Hernandez High School,77.289752,77.289752,80.43253,77.289752
Holden High School,83.803279,83.803279,80.43253,83.803279
Huang High School,76.629414,76.629414,80.43253,76.629414
Johnson High School,77.072464,77.072464,80.43253,77.072464
Pena High School,83.839917,83.839917,80.43253,83.839917


## Reading Score by Grade 

In [24]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
# Same concept as the block of code above.
ninth_graders_scores = school_data_complete.groupby(["school_name"]).mean()
tenth_graders_scores = school_data_complete.groupby(["school_name"]).mean()
eleventh_graders_scores = school_data_complete.groupby(["school_name"]).mean()
twelfth_graders_scores = school_data_complete.groupby(["school_name"]).mean()

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores.mean()["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({"9th": ninth_grade_reading_scores,
                                       "10th": tenth_grader_reading_scores,
                                       "11th": eleventh_grader_reading_scores,
                                       "12th": twelfth_grader_reading_scores})

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.033963,81.033963,82.529188,81.033963
Cabrera High School,83.975780,83.975780,82.529188,83.975780
Figueroa High School,81.158020,81.158020,82.529188,81.158020
Ford High School,80.746258,80.746258,82.529188,80.746258
Griffin High School,83.816757,83.816757,82.529188,83.816757
Hernandez High School,80.934412,80.934412,82.529188,80.934412
Holden High School,83.814988,83.814988,82.529188,83.814988
Huang High School,81.182722,81.182722,82.529188,81.182722
Johnson High School,80.966394,80.966394,82.529188,80.966394
Pena High School,84.044699,84.044699,82.529188,84.044699


## Scores by School Spending

In [25]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [26]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [27]:
# Use `pd.cut` to categorize spending based on the bins.
# The string of code below stating that it will make a new column called "school_spending_df"
# pd.cut is refenrcing the dataset of "per_school_capita (calculated earlier)then wants dataset 
# that will be binned (spending_bins) labels that are assigned to each bin as per index (label)
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels= labels)

In [29]:
#  Calculate averages for the desired columns. 
# Had a bit of complications with this code, soon to realize that the code provided was not referencing
# the same text as what I had typed from the code earlier.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score Per School"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score Per School"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Percent in Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Percent in Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Percent Of Passing Rate"]

In [30]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({"Average Math Score": spending_math_scores,
                                "Average Reading Score": spending_reading_scores,
                                "Percent Passing Math": spending_passing_math,
                                "Percent Passing Reading": spending_passing_reading,
                                "Percent Overall Passing": overall_passing_spending})

# Display results
spending_summary

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-630,81.899826,83.155286,87.133538,92.718205,89.925871
$630-645,78.518855,81.624473,73.484209,84.391793,78.938001
$645-680,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

In [31]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [32]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_counts, size_bins, labels=labels)
per_school_summary

,School Type,Total Students Per School,Total School Budget,Per Student Budget,Average Math Score Per School,Average Reading Score Per School,Percent in Passing Math,Percent in Passing Reading,Percent Of Passing Rate,School Size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992,Large (2000-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270,Small (<1000)


In [33]:
# Calculate averages for the desired columns.
# Same concept as above, again, remember to reference the same text as you defined from earlier.
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score Per School"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score Per School"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["Percent in Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["Percent in Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["Percent Of Passing Rate"]

In [34]:
# Create a DataFrame called `size_summary` that breaks down school performance 
# based on school size (small, medium, or large). Use the scores above to create 
# a new DataFrame called `size_summary`
size_summary = pd.DataFrame({"Average Math Size": size_math_scores,
                            "Average Reading Size": size_reading_scores,
                            "Percent Size Passing Math": size_passing_math,
                            "Percent Size Passing Reading": size_passing_reading,
                            "Percent Size Overall Passing": size_overall_passing})

# Display results
size_summary

,Average Math Size,Average Reading Size,Percent Size Passing Math,Percent Size Passing Reading,Percent Size Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

In [39]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(["School Type"]).mean()
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()
type_passing_math = per_school_summary.groupby(["School Type"]).mean()
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()
type_overall_passing = per_school_summary.groupby(["School Type"]).mean()

# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score Per School"]
average_reading_score_by_type = type_reading_scores["Average Reading Score Per School"]
average_percent_passing_math_by_type = type_passing_math["Percent in Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["Percent in Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["Percent Of Passing Rate"]

In [41]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({"Math Score Per School Type": average_math_score_by_type,
                            "Reading Score Per School Type": average_reading_score_by_type,
                            "Percent Pass Per School Type In Math": average_percent_passing_math_by_type,
                            "Percent Pass Per School Type In Reading": average_percent_passing_reading_by_type,
                            "Overall Percent Pass Per School Type": average_percent_overall_passing_by_type})

# Display results
type_summary

,Math Score Per School Type,Reading Score Per School Type,Percent Pass Per School Type In Math,Percent Pass Per School Type In Reading,Overall Percent Pass Per School Type
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
